In [82]:
import sys
sys.path.append('/Users/celiaberon/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/celiaberon/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import scipy.io as scio
import bandit_preprocessing as bp
import matplotlib.pyplot as plt
import calcium_codes as cc
import hmm_on_behavior as hob
import sys
import os

In [112]:
def alignFrames(record_path, ca_data_path, session_name,mouse_id, cond1, cond2=False, 
                cond3=False, cond1_a=0, cond1_ops='=', cond2_ops='=', cond3_ops='=', 
                extension=30, events=False, display=False, 
                root_dir='/Users/celiaberon/GitHub/mouse_bandit/data/trial_data' ):
    
    record = pd.read_csv(record_path,index_col=0)
    ca_data = scio.loadmat(ca_data_path,squeeze_me = True, struct_as_record = False)
    neuron = ca_data['neuron_results'] 
    
    record[record['Session ID'] == session_name]
    
    """
    Extract data from a specific session
    """
    '''
    load in trial data
    '''
    columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked',
               'Right Reward Prob','Left Reward Prob','Reward Given',
              'center_frame','decision_frame']

    #root_dir = '/Users/celiaberon/GitHub/mouse_bandit/data/trial_data'

    full_name = session_name + '_trials.csv'

    path_name = os.path.join(root_dir,full_name)

    trial_df = pd.read_csv(path_name,names=columns)
    
    """
    Convert to feature matrix
    """
    
    feature_matrix = bp.create_feature_matrix(trial_df,10,mouse_id,session_name,feature_names='Default',imaging=True)
    
    beliefs_feat_mat = hob.predictBeliefFeatureMat(feature_matrix, 10)

    feature_matrix['Belief'] = beliefs_feat_mat
    
    """
    Define function to get frames based on up to 3 conditions
    """
    
    def extract_frames(df, cond1_name, cond1=False, cond2_name=False,cond2=False, cond3_name=False,
                       cond3=False, cond1_ops='=', cond2_ops ='=', cond3_ops ='='):

        import operator

        # set up operator dictionary
        ops = {'>': operator.gt,
           '<': operator.lt,
           '>=': operator.ge,
           '<=': operator.le,
           '=': operator.eq}

        if type(cond3_name)==str:
            frames_c = (df[((ops[cond1_ops](df[cond1_name],cond1)) 
                        & (ops[cond2_ops](df[cond2_name], cond2))
                        & (ops[cond3_ops](df[cond3_name],cond3)))]['center_frame'])
            frames_d = (df[((ops[cond1_ops](df[cond1_name],cond1)) 
                        & (ops[cond2_ops](df[cond2_name], cond2))
                        & (ops[cond3_ops](df[cond3_name],cond3)))]['decision_frame'])
            frames = np.column_stack((frames_c, frames_d))
            return frames

        elif type(cond2_name)==str:
            frames_c = (df[((ops[cond1_ops](df[cond1_name],cond1)) 
                        & (ops[cond2_ops](df[cond2_name],cond2)))]['center_frame'])
            frames_d = (df[((ops[cond1_ops](df[cond1_name],cond1)) 
                        & (ops[cond2_ops](df[cond2_name],cond2)))]['decision_frame'])
            frames = np.column_stack((frames_c, frames_d))
            return frames

        else:
            frames_c =(df[(df[cond1_name] == cond1)]['center_frame'])
            frames_d =(df[(df[cond1_name] == cond1)]['decision_frame'])
            frames = np.column_stack((frames_c, frames_d))
            return frames
    
    """
    Set the parameters to input into extract_frames function
    """
    
    cond1_name = cond1
    #cond1_a = 0
    cond1_b = 1-cond1_a
    cond2_name = cond2
    cond2_a = 0
    cond2_b = 1
    cond3_name = cond3
    cond3_a = 0
    cond3_b = 1
    
    cond1_ops_b = '=' # default value for cond1_ops_b
    if cond1_ops != '=':
        if cond1_ops == '>':
            cond1_ops_b = '<='
        elif cond1_ops == '>=':
            cond1_ops_b = '<'

    conditions = [cond1_name, cond2_name, cond3_name]
    if type(cond3)==str:
        n_variables = 3
    elif type(cond2)==str:
        n_variables = 2
    else:
        n_variables = 1
            
    # center frames in first column, decision frames in second
    fr_1a2a3a = extract_frames(feature_matrix, cond1_name, cond1_a, 
                               cond2_name, cond2_a, cond3_name, cond3_a, cond1_ops=cond1_ops)

    fr_1b2a3a = extract_frames(feature_matrix, cond1_name, cond1_b, 
                               cond2_name, cond2_a, cond3_name, cond3_a, cond1_ops=cond1_ops_b)

    fr_1a2b3a = extract_frames(feature_matrix, cond1_name, cond1_a, 
                               cond2_name, cond2_b, cond3_name, cond3_a, cond1_ops=cond1_ops)

    fr_1b2b3a = extract_frames(feature_matrix, cond1_name, cond1_b, 
                               cond2_name, cond2_b, cond3_name, cond3_a, cond1_ops=cond1_ops_b)

    fr_1a2b3b = extract_frames(feature_matrix, cond1_name, cond1_a, 
                               cond2_name, cond2_b, cond3_name, cond3_b, cond1_ops=cond1_ops)

    fr_1a2a3b = extract_frames(feature_matrix, cond1_name, cond1_a, 
                               cond2_name, cond2_a, cond3_name, cond3_b, cond1_ops=cond1_ops)

    fr_1b2a3b = extract_frames(feature_matrix, cond1_name, cond1_b, 
                               cond2_name, cond2_a, cond3_name, cond3_b, cond1_ops=cond1_ops_b)

    fr_1b2b3b = extract_frames(feature_matrix, cond1_name, cond1_b, 
                               cond2_name, cond2_b, cond3_name, cond3_b, cond1_ops=cond1_ops_b)

    var_keys = '1a2a3a', '1b2a3a', '1a2b3a', '1b2b3a', '1a2b3b', '1a2a3b', '1b2a3b', '1b2b3b'
    groupings_2 = np.stack(((0,5), (1,6), (2,4), (3,7)))
    groupings_1 = np.stack(((0,2,4,5), (1,3,6,7)))
    
    
    n_combos = 2**n_variables

    for i in range(n_combos):
        if n_variables == 3:
            if i == 0:
                start_stop_frames = {var_keys[i]:eval('fr_%s' %var_keys[i])}
            if i > 0:
                start_stop_frames.update({var_keys[i]:eval('fr_%s' %var_keys[i])})
        if n_variables == 2:
            if i == 0:
                start_stop_frames = {var_keys[i][0:4]: np.transpose(np.column_stack((
                            np.transpose(eval('fr_%s' % var_keys[groupings_2[i][0]])),
                            np.transpose(eval('fr_%s' % var_keys[groupings_2[i][1]])))))}
            if i > 0:
                start_stop_frames.update({var_keys[i][0:4]: np.transpose(np.column_stack((
                            np.transpose(eval('fr_%s' % var_keys[groupings_2[i][0]])),
                            np.transpose(eval('fr_%s' % var_keys[groupings_2[i][1]])))))})
                if i == np.max(n_combos)-1:
                    var_keys = list(start_stop_frames.keys())  

        if n_variables == 1:
            if i == 0:
                start_stop_frames = {var_keys[i][0:2]: np.transpose(np.column_stack((
                            np.transpose(eval('fr_%s' % var_keys[groupings_1[i][0]])),
                            np.transpose(eval('fr_%s' % var_keys[groupings_1[i][1]])),
                            np.transpose(eval('fr_%s' % var_keys[groupings_1[i][2]])),
                            np.transpose(eval('fr_%s' % var_keys[groupings_1[i][3]])))))}
            if i > 0:
                start_stop_frames.update({var_keys[i][0:2]: np.transpose(np.column_stack((
                            np.transpose(eval('fr_%s' % var_keys[groupings_1[i][0]])),
                            np.transpose(eval('fr_%s' % var_keys[groupings_1[i][1]])),
                            np.transpose(eval('fr_%s' % var_keys[groupings_1[i][2]])),
                            np.transpose(eval('fr_%s' % var_keys[groupings_1[i][3]])))))})
                if i == np.max(n_combos)-1:
                    var_keys = list(start_stop_frames.keys())            

    for i in start_stop_frames:
        start_stop_frames[i][:,0] = start_stop_frames[i][:,0] - extension
        start_stop_frames[i][:,1] = start_stop_frames[i][:,1] + extension
        
    if events == True:
        events = cc.detectEvents(ca_data_path)

        neuron.C_raw = np.copy(events)
        nNeurons = neuron.C_raw.shape[0]
        nFrames = neuron.C_raw.shape[1]

        #Create Gaussian filter and apply to raw trace
        sigma = 1.5;
        sz = 5;  

        x = np.linspace(-sz / 2, sz / 2, sz);
        gaussFilter = np.exp(-x**2 / (2*sigma**2));
        gaussFilter = gaussFilter / np.sum(gaussFilter);

        smoothed = np.zeros((nNeurons, neuron.C_raw.shape[1]+sz-1));

        for i in range(0, nNeurons):
            smoothed[i,:] = np.convolve(neuron.C_raw[i,:], gaussFilter);
            
        neuron.C_raw = smoothed[:,0:nFrames]
        
    nTrials = []; window_length=[]
    max_window = np.zeros(n_combos)

    for i in range(n_combos):
        nTrials.append(start_stop_frames[var_keys[i]].shape[0])

        for iTrial in range(nTrials[i]):
            window_length.append((start_stop_frames[var_keys[i]][iTrial][1]-
                                     start_stop_frames[var_keys[i]][iTrial][0]))
        max_window[i] = np.max(window_length)
    
    max_window = int(max_window.max())
    
    aligned_start = np.zeros((np.max(nTrials), max_window, nNeurons, n_combos))
    mean_center_poke = np.zeros((max_window, nNeurons, n_combos))

    for i in range(n_combos):

        # create array containing segment of raw trace for each neuron for each trial 
        # aligned to center poke
        for iNeuron in range(nNeurons):
            for iTrial in range(0,nTrials[i]):
                aligned_start[iTrial,:, iNeuron, i] = neuron.C_raw[iNeuron,
                    int(start_stop_frames[var_keys[i]][iTrial][0]):
                    (int(start_stop_frames[var_keys[i]][iTrial][0])+max_window)]

        # take mean of fluorescent traces across all trials for each neuron, then normalize for each neuron
        mean_center_poke[:,:,i]= np.mean(aligned_start[0:nTrials[i],:,:,i], axis=0)
        
    aligned_decision = np.zeros((np.max(nTrials), max_window, nNeurons, n_combos))
    mean_decision = np.zeros((max_window, nNeurons, n_combos))

    for i in range(n_combos):

        # create array containing segment of raw trace for each neuron for each trial 
        # aligned to decision poke
        for iNeuron in range(nNeurons):
            for iTrial in range(nTrials[i]):
                aligned_decision[iTrial,:, iNeuron, i] = neuron.C_raw[iNeuron, 
                    int(start_stop_frames[var_keys[i]][iTrial][1])-max_window:
                    (int(start_stop_frames[var_keys[i]][iTrial][1]))]

        # take mean of fluorescent traces across all trials for each neuron
        mean_decision[:,:,i]= np.mean(aligned_decision[0:nTrials[i],:,:,i], axis=0)
    
    if display:
        
        ydim = n_combos/2
        plt.figure(figsize=(8,ydim*4))
        for i in range(n_combos):

            plt.subplot(ydim,2,i+1)  
            plt.imshow(np.transpose(mean_center_poke[:,:,i])), plt.colorbar()
            plt.axvline(x=extension, color='k', linestyle = '--', linewidth=.9)
            plt.axis('tight')
            plt.xlabel('Frame (center poke at %s)' % extension)
            plt.ylabel('Neuron ID')
            if n_variables == 3:
                plt.title("%s = %s\n %s = %s\n%s = %s\nNum trials = %.0f" 
                          %(conditions[int(var_keys[i][0])-1], var_keys[i][1],
                            conditions[int(var_keys[i][2])-1], var_keys[i][3], 
                            conditions[int(var_keys[i][4])-1],
                            var_keys[i][5], nTrials[i])) 
            if n_variables == 2:
                plt.title("%s = %s\n %s = %s\nNum trials = %.0f" 
                          %(conditions[int(var_keys[i][0])-1], var_keys[i][1],
                            conditions[int(var_keys[i][2])-1], var_keys[i][3], nTrials[i]))
            if n_variables == 1:
                plt.title("%s = %s\nNum trials = %.0f" 
                          %(conditions[int(var_keys[i][0])-1], var_keys[i][1], nTrials[i]))

        plt.tight_layout()
            
        plt.figure(figsize=(8, ydim*4))
        for i in range(n_combos):
            plt.subplot(ydim,2,i+1)  
            plt.imshow(np.transpose(mean_decision[:,:,i])), plt.colorbar()
            plt.axvline(x=max_window-extension, color='k', linestyle = '--', linewidth=.9)
            plt.xlabel('Frames (decision poke at %s)' % (max_window-extension))
            plt.ylabel('Neuron ID')
            plt.axis('tight')
            if n_variables == 3:
                plt.title("%s = %s\n %s = %s\n%s = %s\nNum trials = %.0f" 
                          %(conditions[int(var_keys[i][0])-1], var_keys[i][1],
                            conditions[int(var_keys[i][2])-1], var_keys[i][3], 
                            conditions[int(var_keys[i][4])-1],
                            var_keys[i][5], nTrials[i])) 
            if n_variables == 2:
                plt.title("%s = %s\n %s = %s\nNum trials = %.0f" 
                          %(conditions[int(var_keys[i][0])-1], var_keys[i][1],
                            conditions[int(var_keys[i][2])-1], var_keys[i][3], nTrials[i]))
            if n_variables == 1:
                plt.title("%s = %s\nNum trials = %.0f" 
                          %(conditions[int(var_keys[i][0])-1], var_keys[i][1], nTrials[i]))
        plt.tight_layout()
            
    return neuron.C_raw, mean_center_poke, mean_decision, var_keys

In [111]:
record_path = '/Users/celiaberon/GitHub/mouse_bandit/session_record.csv'
ca_data_path = '/Volumes/Neurobio/MICROSCOPE/Celia/data/k7_03142017_test/neuron_results.mat'
session_name = '03142017_K7'
mouse_id = 'K7'
cond1 = 'Belief'
cond2 = 'Decision'
cond3 = 'Reward'

[neuron, mean_center_poke, mean_decision, var_keys] = alignFrames(record_path, ca_data_path,
                session_name, mouse_id, cond1, cond2, cond1_a = 0,
                events=True, display=True)

ValueError: zero-size array to reduction operation maximum which has no identity